In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import io
import sys
import json

import numpy as np

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import download_file, clear_download_cache

import matplotlib.pyplot as plt

## Query HSC database using SQL search

* Here `unagi` provides the basic function to run quick SQL search of HSC database
* If you need to run large `SQL` search (longer than ~10 mins), please use the `CAS` online search interface

### Step 1: Setup HSC-SSP online data archive 

* First, you need to setup a HSC-SSP rerun
* Here we use the wide field from the `PDR1` as an example
* `SQL` search only cares about the data release (`dr`), you can use any `rerun` you want and still have access to all 

In [10]:
from unagi import config
from unagi import hsc
from unagi import plotting
from unagi.task import hsc_cutout

# TODO need to change to PDR
pdr1 = hsc.Hsc(dr='pdr1', rerun='pdr1_wide')

### Basic SQL search 

* You need to write your own `SQL` script and pass it to query
* The output is a `astropy.table` table. It can be saved as an output file
* By default
    - We will skip the syntax check.  If you want syntax check, please set `skip_syntax=False`
    - We will delete the search result from the database after the search is done. If you want to keep the data, please set `delete_after=False`
    - We turn off the e-mail notification. If you don't mind being bothered by a lot of e-mails, set `noemail=False`

In [11]:
sql_test = open('../unagi/test/test_query_1.sql', 'r').read()
print(sql_test)

SELECT 
        object_id 
      , ra 
      , dec 
      , imag_kron 
      , imag_kron_err 
      , ymag_kron 
      , ymag_kron_err 
      , imag_kron - ymag_kron AS i_y 
    FROM 
        pdr1_udeep.forced 
    WHERE 
        boxsearch(coord, 34.0, 36.0, -5.0, -4.5) 
        /* is equivalent to 
             ra BETWEEN 34.0 AND 36.0 
             AND dec BETWEEN -5.0 AND -4.5 
           but boxSearch() is much faster 
        */ 
        AND imag_kron < 25.5 
    LIMIT 10
;


In [12]:
result_test = pdr1.sql_query(sql_test, verbose=True)

print(result_test)

Waiting for query to finish... [Done]
    object_id             ra         ... ymag_kron_err     i_y     
----------------- ------------------ ... ------------- ------------
37484855356823644 34.497888478553904 ...   0.011460446   0.49466515
37484855356851947  34.49811385510532 ...    0.16973113    0.7528496
37484855356852577 34.499461064481956 ...    0.16208024    0.2631302
37484855356853110  34.49828081308036 ...    0.20196179   0.10679436
37484855356853409  34.49914189505791 ...   0.038959116   0.45888138
37484855356853410  34.49971429719763 ...    0.06631608  0.041065216
37484855356823966  34.49709026091776 ...    0.21807384  -0.47187042
37484855356824754 34.496511370237826 ...   0.020181475   0.43512917
37484855356825704  34.49579931726271 ...    0.15489967 -0.066129684
37484855356851943  34.49745850750885 ...   0.042136233    1.0453167


### Preview a large SQL search

* You can also test certain `SQL` search using the `preview=True` option. It will return the first 10 objects.

In [35]:
res = pdr1.sql_query(sql_test, preview=True, verbose=True)

# Preview the SQL search result...
['object_id', 'ra', 'dec', 'imag_kron', 'imag_kron_err', 'ymag_kron', 'ymag_kron_err', 'i_y']
['37484855356823644', '34.4978884785539037', '-4.63005420066825835', '22.0537434', '0.0050310907', '21.5590782', '0.0114604458', '0.494665146']
['37484855356851947', '34.4981138551053235', '-4.62716435375749313', '25.120718', '0.0899058282', '24.3678684', '0.169731125', '0.752849579']
['37484855356852577', '34.4994610644819559', '-4.62184521677133997', '24.9699688', '0.0547340661', '24.7068386', '0.162080243', '0.263130188']
['37484855356853110', '34.4982808130803633', '-4.61936769456526353', '25.2625847', '0.0585449859', '25.1557903', '0.201961786', '0.106794357']
['37484855356853409', '34.4991418950579103', '-4.61608885085157006', '23.2742653', '0.0153167928', '22.8153839', '0.0389591157', '0.458881378']
['37484855356853410', '34.4997142971976274', '-4.61655727777253944', '23.7624779', '0.0178093165', '23.7214127', '0.0663160831', '0.0410652161']
['37484855

In [ ]:
### 